In [4]:
!which pyspark 

/workspaces/Real-Estate-Analyze/spark-3.4.3-bin-hadoop3/bin/pyspark


In [15]:
!pip install logging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for logging: filename=logging-0.4.9.6-py3-none-any.whl size=29516 sha256=31859d913aa7acc74ee386a2bec235b65fbc54d1509640f5a8abb6f27d88a123
  Stored in directory: /home/codespace/.cache/pip/wheels/c2/46/de/9ff6b7043ccdbac300a6f5b61d6220c8f2e9984f7abbf53d96
Successfully built logging

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
pip install findspark

  Obtaining dependency information for findspark from https://files.pythonhosted.org/packages/a4/cb/7d2bb508f4ca00a043fd53e8156c11767799d3f534bf451a0942211d5def/findspark-2.0.1-py2.py3-none-any.whl.metadata
  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("streamData").master("local[*]").config("spark.executor.memory", "8g").config("spark.executor.cores", "4").getOrCreate()

24/10/02 14:15:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
INPUT_DATA_PATH = 'hdfs://127.0.0.1:9000/user/stream_data/stream_data/'
FINAL_OUTPUT_PATH = 'hdfs://127.0.0.1:9000/user/stream_data/out/'

In [6]:

def get_hdfs_file_system():
    try:
        uri = spark.sparkContext._gateway.jvm.java.net.URI(INPUT_DATA_PATH)
        conf = spark.sparkContext._jsc.hadoopConfiguration()
        FileSystem = spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.FileSystem
        return FileSystem.get(uri, conf)
    except Exception as e:
        print(f"Error getting HDFS FileSystem: {e}")
        return None
fs = get_hdfs_file_system()

In [8]:

def list_hdfs_directories(fs, path):
    try:
        status = fs.listStatus(spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path(path))
        return [file.getPath().getName() for file in status if file.isDirectory()]
    except Exception as e:
        print(f"Error listing HDFS directories: {e}")
        return []
dirs = list_hdfs_directories(fs,INPUT_DATA_PATH) 

In [11]:

def list_hdfs_files(fs, path):
    try:
        status = fs.listStatus(spark.sparkContext._gateway.jvm.org.apache.hadoop.fs.Path(path))
        return [file.getPath().getName() for file in status if file.isFile()]
    except Exception as e:
        print(f"Error listing HDFS files: {e}")
        return []
files = list_hdfs_files(fs, INPUT_DATA_PATH)

In [12]:
for dirname in dirs:
        data_path = f"{INPUT_DATA_PATH}/{dirname}"
        files = list_hdfs_files(fs, data_path)
        print(files)
        

['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
['co2.

In [23]:
    dataframes = {}
    dataframes_room = {}
    columns = ['co2', 'humidity', 'light', 'pir', 'temperature']
    dirs = list_hdfs_directories(fs, INPUT_DATA_PATH)
    
    print(f"Directories found: {dirs}")

    for dirname in dirs:
        data_path = f"{INPUT_DATA_PATH}/{dirname}"
        files = list_hdfs_files(fs, data_path)
        
        print(f"Files found in {dirname}: {files}")
        
        count = 0
        for filename in files:
            try:
                filepath = f"{data_path}/{filename}"
                print(filepath)
                data_filename = f"{dirname}_{filename.split('.')[0]}"
                print(data_filename)
                df = spark.read.csv(filepath, header=True, inferSchema=True)
                print(df)
                df = df.toDF('ts_min_bignt', columns[count])
                print(df)
                dataframes[data_filename] = df
                print(df)
                # print(f"DataFrame created for {data_filename}")
                # count += 1
            except Exception as e:
                print(f"Error reading file {filename} in directory {dirname}: {e}")


Directories found: ['413', '415', '417', '419', '421', '422', '423', '424', '442', '446', '448', '452', '454', '456', '458', '462', '510', '511', '513', '552', '554', '556', '558', '562', '564', '621', '621A', '621C', '621D', '621E', '640', '644', '648', '656A', '656B', '664', '666', '668', '717', '719', '721', '722', '723', '724', '726', '734', '746', '748', '752', '754', '776']
Files found in 413: ['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
hdfs://127.0.0.1:9000/user/stream_data/stream_data//413/co2.csv
413_co2
DataFrame[1377299107: int,  488.00: double]
DataFrame[ts_min_bignt: int, co2: double]
DataFrame[ts_min_bignt: int, co2: double]
Files found in 415: ['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
hdfs://127.0.0.1:9000/user/stream_data/stream_data//415/co2.csv
415_co2
DataFrame[1377275058: int,  462.00: double]
DataFrame[ts_min_bignt: int, co2: double]
DataFrame[ts_min_bignt: int, co2: double]
Files found in 417: ['co2.csv', 'hu

In [26]:

def create_separate_dataframes(fs) -> dict:
    """
    Creates a dictionary that includes room numbers as keys and dataframes per room as values
    """
    dataframes = {}
    dataframes_room = {}
    columns = ['co2', 'humidity', 'light', 'pir', 'temperature']
    dirs = list_hdfs_directories(fs, INPUT_DATA_PATH)
    
    print(f"Directories found: {dirs}")

    for dirname in dirs:
        data_path = f"{INPUT_DATA_PATH}/{dirname}"
        files = list_hdfs_files(fs, data_path)
        
        print(f"Files found in {dirname}: {files}")
        
        count = 0
        for filename in files:
            try:
                filepath = f"{data_path}/{filename}"
                data_filename = f"{dirname}_{filename.split('.')[0]}"
                df = spark.read.csv(filepath, header=True, inferSchema=True)
                df = df.toDF('ts_min_bignt', columns[count])
                dataframes[data_filename] = df
                print(f"DataFrame created for {data_filename}")
                count += 1
            except Exception as e:
                print(f"Error reading file {filename} in directory {dirname}: {e}")
        
        try:
            dataframes[f'{dirname}_co2'].createOrReplaceTempView('df_co2')
            dataframes[f'{dirname}_humidity'].createOrReplaceTempView('df_humidity')
            dataframes[f'{dirname}_light'].createOrReplaceTempView('df_light')
            dataframes[f'{dirname}_pir'].createOrReplaceTempView('df_pir')
            dataframes[f'{dirname}_temperature'].createOrReplaceTempView('df_temperature')

            dataframes_room[dirname] = spark.sql('''
                SELECT
                  df_co2.*,
                  df_humidity.humidity,
                  df_light.light,
                  df_pir.pir,
                  df_temperature.temperature        
                FROM df_co2
                INNER JOIN df_humidity
                  ON df_co2.ts_min_bignt = df_humidity.ts_min_bignt
                INNER JOIN df_light
                  ON df_humidity.ts_min_bignt = df_light.ts_min_bignt
                INNER JOIN df_pir
                  ON df_light.ts_min_bignt = df_pir.ts_min_bignt
                INNER JOIN df_temperature
                  ON df_pir.ts_min_bignt = df_temperature.ts_min_bignt      
            ''')
            dataframes_room[dirname] = dataframes_room[dirname].withColumn("room", F.lit(dirname))
            print(f"DataFrames for room {dirname} successfully combined")
        except KeyError as e:
            print(f"KeyError: {e} - Check if all data files are present for {dirname}")
        except AnalysisException as e:
            print(f"AnalysisException: {e} - Issue creating SQL view for {dirname}")
    
    return dataframes_room


In [28]:
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F

create_separate_dataframes(fs)

Directories found: ['413', '415', '417', '419', '421', '422', '423', '424', '442', '446', '448', '452', '454', '456', '458', '462', '510', '511', '513', '552', '554', '556', '558', '562', '564', '621', '621A', '621C', '621D', '621E', '640', '644', '648', '656A', '656B', '664', '666', '668', '717', '719', '721', '722', '723', '724', '726', '734', '746', '748', '752', '754', '776']
Files found in 413: ['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
DataFrame created for 413_co2
DataFrame created for 413_humidity
DataFrame created for 413_light
DataFrame created for 413_pir
DataFrame created for 413_temperature
DataFrames for room 413 successfully combined
Files found in 415: ['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
DataFrame created for 415_co2
DataFrame created for 415_humidity
DataFrame created for 415_light
DataFrame created for 415_pir
DataFrame created for 415_temperature
DataFrames for room 415 successfully combined
Files found i

{'413': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '415': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '417': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '419': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '421': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '422': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '423': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '424': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: d

In [29]:
def merge_dataframes(dfs):
    """
    Merges multiple dataframes vertically.
    """
    try:
        return reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)
    except Exception as e:
        print(f"Error in merge_dataframes: {e}")
        return None

def create_main_dataframe(separate_dataframes: dict):
    """
    Merges all per-room dataframes vertically. Creates final dataframe.
    """
    try:
        # Collect all DataFrames
        dataframes_to_concat = list(separate_dataframes.values())
        
        # Merge DataFrames
        df = merge_dataframes(dataframes_to_concat)
        df = df.sort(F.col("ts_min_bignt"))

        df = df.dropna()

        # Convert ts_min_bignt to event_ts_min correctly
        df = df.withColumn("event_ts_min", F.from_unixtime(F.col("ts_min_bignt")).cast("timestamp"))
        df = df.withColumn("event_ts_min", F.date_format(F.col("event_ts_min"), "yyyy-MM-dd HH:mm:ss"))

        print("Successfully created main dataframe.")
        return df
    except Exception as e:
        print(f"Error in create_main_dataframe: {e}")
        return None


In [30]:

def create_main_dataframe(separate_dataframes: dict):
    """
    Merges all per-room dataframes vertically. Creates final dataframe.
    """
    try:
        # Collect all DataFrames
        dataframes_to_concat = list(separate_dataframes.values())
        
        # Merge DataFrames
        df = merge_dataframes(dataframes_to_concat)
        df = df.sort(F.col("ts_min_bignt"))

        df = df.dropna()

        # Convert ts_min_bignt to event_ts_min correctly
        df = df.withColumn("event_ts_min", F.from_unixtime(F.col("ts_min_bignt")).cast("timestamp"))
        df = df.withColumn("event_ts_min", F.date_format(F.col("event_ts_min"), "yyyy-MM-dd HH:mm:ss"))

        print("Successfully created main dataframe.")
        return df
    except Exception as e:
        print(f"Error in create_main_dataframe: {e}")
        return None


In [31]:
room_dataframes = create_separate_dataframes(fs)
room_dataframes

Directories found: ['413', '415', '417', '419', '421', '422', '423', '424', '442', '446', '448', '452', '454', '456', '458', '462', '510', '511', '513', '552', '554', '556', '558', '562', '564', '621', '621A', '621C', '621D', '621E', '640', '644', '648', '656A', '656B', '664', '666', '668', '717', '719', '721', '722', '723', '724', '726', '734', '746', '748', '752', '754', '776']
Files found in 413: ['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
DataFrame created for 413_co2
DataFrame created for 413_humidity
DataFrame created for 413_light
DataFrame created for 413_pir
DataFrame created for 413_temperature
DataFrames for room 413 successfully combined
Files found in 415: ['co2.csv', 'humidity.csv', 'light.csv', 'pir.csv', 'temperature.csv']
DataFrame created for 415_co2
DataFrame created for 415_humidity
DataFrame created for 415_light
DataFrame created for 415_pir
DataFrame created for 415_temperature
DataFrames for room 415 successfully combined
Files found i

{'413': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '415': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '417': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '419': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '421': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '422': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '423': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: double, pir: double, temperature: double, room: string],
 '424': DataFrame[ts_min_bignt: int, co2: double, humidity: double, light: d

In [ ]:

def write_dataframe(df):
    """
    Writes the final dataframe to HDFS in CSV format.
    """
    try:
        df.write.parquet(FINAL_OUTPUT_PATH + "sensors.parquet", mode='overwrite')
        print("Successfully written dataframe to HDFS!")
    except Exception as e:
        print(f"Error in write_dataframe: {e}")

if __name__ == "__main__":
    fs = get_hdfs_file_system()
    if fs:
        room_dataframes = create_separate_dataframes(fs)
        if room_dataframes:
            main_df = create_main_dataframe(room_dataframes)
            if main_df:
                write_dataframe(main_df)
